### Import

---

In [1]:
import numpy as np
from note_include.elem.LWE    import LWE
from note_include.utils.types import LWEctxt

np.set_printoptions(threshold=np.inf, linewidth=np.inf)

---

# $\textsf{Key Switch and Mod Switch}$

* <a href="https://eprint.iacr.org/2014/816"> FHEW: Bootstrapping Homomorphic Encryption in less than a second </a>

* <a href="https://eprint.iacr.org/2020/086"> Bootstrapping in FHEW-like Cryptosystems </a>

---

FHEW 의 꽃인 blind rotation 을 서술하기 앞서 함께 사용되는 부가적인 알고리즘인 key switch 와 modulus switch 에 대해서 알아보자.

우선 key switch 부터 시작하자.

---

## 1. Key switch

Key switch 는 FHEW 외에도 다양한 동형암호 스킴에서 자주 사용되는 테크닉이다. 

동형연산을 통해 암호화 된 상태에서 비밀키를 바꾸어주는 테크닉이며 동형 연산으로 수행하기 때문에 노이즈를 발생시킨다.

시작하기 앞서 다음을 생각해보자. LWE 암호를 다음과 같이 표현할 수 있다.

$$
    \textsf{LWE}_{\vec{s}}(m) = \langle \vec{a}, \vec{s} \rangle + m + e.
$$

이 상황에서 우리는 $\vec{s}$ 로 암호화 되어있는 암호문을 메세지는 유지한 채로 $\vec{s}'$ 으로 암호화 하고 싶은 것 이다.

$$
    \textsf{LWE}_{\vec{s'}}(m) = \langle \vec{a}', \vec{s}' \rangle + m + e.
$$

$\textsf{LWE}_{\vec{s}}(m)$ 을 $\textsf{LWE}_{\vec{s'}}(m)$ 으로 바꾸어주기 위해서 우리는 무엇을 수행해야 하는지 고민해보자. 다음과 같은 연산을 수행하면 된다는 것을 금방 알 수 있다.

\begin{align*}
    \textsf{LWE}_{\vec{s}}(m) + \textsf{LWE}_{\vec{s'}}(-\langle \vec{a}, \vec{s} \rangle)
    &=  \langle \vec{a}, \vec{s} \rangle + m + e_0 + \langle \vec{a}', \vec{s}' \rangle -\langle \vec{a}, \vec{s} \rangle + e_1 \\
    &= \langle \vec{a}', \vec{s}' \rangle + m + e_0 + e_1.
\end{align*}

여기서 key switching 에 사용되는 암호문 $\textsf{LWE}_{\vec{s'}}(-\langle \vec{a}, \vec{s} \rangle)$ 을 key switching key 라고 칭한다.

그러나 이 방식 같은 경우에는 문제점이 있다. $\textsf{LWE}_{\vec{s'}}(-\langle \vec{a}, \vec{s} \rangle)$ 을 만들어야 하는데 $\vec{a}$ 의 경우에는 정해져 있는 값이 아니라 연산을 수행함에 따라 계속해서 바뀌기 때문이다.

무식하게 모든 $\langle \vec{a}, \vec{s} \rangle$ 에 대해서 전부 만들어둔다면 암호문의 수가 대략 $\mathcal{O}(q^n \times n)$ 가 되어 엄청 커지게 될 것이다.

이를 피하기 위해 일반적으로는 모든 $a \in \mathbb{Z}_q$ 를 만들 수 있도록 한다.

앞서 배웠던 gadget decomposition 과 매우 유사하다. 예를 들어 $\mathbb{Z}_q$ 에서 정의되는 임의의 값은 gadget decomposition 을 통해 다음과 같이 표현할 수 있다.

\begin{align*}
    a = \sum_{i=0}^{d-1} v_i B^i, \text{ where } v_i \in \mathbb{Z}_B \text{ and } d = \lceil \log_Bq \rceil.
\end{align*}

여기서 $(v_i) = \textsf{gadget\_decomposition\_int}(a, B)$ 이다.

즉, 임의의 $a$ 를 $\sum_{i=0}^{d-1} \text{LWE}_{\vec{s}'}(v_i B^i)$ 형식으로 만들 수 있다는 것이다.

결론적으로 key switching 에 사용되는 key switching key 는 다음과 같이 정의할 수 있다.

>$\textbf{Definition : Key siwtching key}$
>
>어떠한 base value $B_\text{ks}$ 와 두 비밀키 $\vec{s}, \vec{s}'$ 이 있을 때, key switching key $\textsf{ksk}$ 는 다음과 같이 정의할 수 있다.
>
>\begin{align*}
>    \textsf{ksk}_{i, j, v} = \left\{ \text{LWE}_{\vec{s'}} \left( v B_\text{ks}^j s_i \right) | i \in [0, n), j \in [0, d_\text{ks}), v \in [0, B_\text{ks})  \right \} 
>\end{align*}
>
>여기서 $d_\text{ks} = \lceil \log_{B_\text{ks}}q \rceil$ 이다.

이를 통해 기존의 $\mathcal{O}(q^n \times n)$ 을 $\mathcal{O}(d_\text{ks} \times B_\text{ks} \times n)$ 으로 줄일 수 있다.

코드를 통해 확인해보자.


In [2]:
def KSKgen(lwe_sk:list[int], lwe_sk2:list[int], B:int, d:int, lweCC:LWE):
    q   = lweCC.q
    ksk = []
    for _s in lwe_sk2: # 버리는 비밀키
        matrix = []
        for v in range(B):
            row = []
            for j in range(d):
                row.append(lweCC.encrypt((v * (B**j) * _s) % q, lwe_sk)) # 새로 얻는 비밀키
            matrix.append(row)
        ksk.append(matrix)    
    return ksk


---

이어서 key switching 은 다음과 같이 정의할 수 있다.

>$\textbf{Definition : Key Switch}$
>
>비밀키 $\vec{s}$ 로 암호화 되어있는 LWE 암호문 $(\vec{a}, b) \in \mathbb{Z}_q^n$ 이 있다고 하자. $\textsf{ksk}_{i, j, v}$ 를 key switching key 라 할 때 key switching 은 다음과 같이 정의할 수 있다:
>
>\begin{align*}
>    \textsf{KeySwitch}((\vec{a}, b), \textsf{ksk}) = (0, b) - \sum_{a_i} \textsf{ksk}_{i, j, a_{ij}}, \text{ where } a_{ij} \in \mathbb{Z}_{B_\text{ks}}.
>\end{align*}

코드를 통해 확인해보자.

In [3]:
def KeySwitch(ct:LWEctxt, ksk:list[list[LWEctxt]], B:int, d:int, lweCC:LWE) -> LWEctxt:
    a, b = ct
    tmp_ct = (np.zeros(lweCC.n), b)
    for i, _a in enumerate(a):
        tmpa = int(_a)
        for j in range(d):
            v  = tmpa % B
            tmpa = tmpa // B

            tmp_ct = lweCC.sub(tmp_ct, ksk[i][v][j])
    
    return tmp_ct

In [4]:
n   = 16
q   = 2048
B   = 128
d   = int(np.ceil(np.log(q) / np.log(B)))
std = 1.2

lwe = LWE(n, q, s_std="Gaussian", e_std=std)
m     = 400
s1  = lwe.keygen() # 버릴 비밀키
s2  = lwe.keygen() # 새로 얻는 비밀키
ksk = KSKgen(s2, s1, B, d, lwe)

ct_s1 = lwe.encrypt(m, s1)
ct_s2 = KeySwitch(ct_s1, ksk, B, d, lwe)

pt_s1 = lwe.decrypt(ct_s2, s1)
pt_s2 = lwe.decrypt(ct_s2, s2)

print("Message                     : ", m)
print("Decrypt with old secret key : ", pt_s1)
print("Decrypt with new secret key : ", pt_s2)

Message                     :  400
Decrypt with old secret key :  1498.0
Decrypt with new secret key :  382.0



---

약간의 오차를 보이지만 정상적으로 작동하는 모습을 확인할 수 있다. 

이전 비밀키를 사용할 경우 정상적인 복호화가 불가능한 것 또한 확인할 수 있다.

이번에는 modulus switching 에 대해서 알아보도록 하자.

## 2. Modulus switch

FHEW 는 재밌게도 LWE 암호문과 RLWE 암호문을 전부 사용한다. 즉, 파라미터를 n, q, N, Q 로 사용한다는 것이다.

후에 배울 blind rotation 을 큰 관점에서 보게되면 다음과 같은 흐름을 갖는다.

$$
\text{LWE} \longrightarrow \text{RLWE} \longrightarrow \text{LWE}.
$$

즉, 암호문을 LWE 에서 RLWE 로 바꾸고, 그 다음 다시 LWE 로 되돌리는 과정을 거치게 된다.

이 때, LWE 와 RLWE 의 modulus $q,Q$ 는 서로 다르므로 이를 switch 하는 동작을 필요로 하게 된다.

이에 따라 modulus switch 를 정의하여 사용하는 것이며 이는 다음과 같이 정의할 수 있다:

>$\textbf{Definition : Modulus Switch}$
>
>LWE 암호문 $(\vec{a}, b) \in \mathbb{Z}_Q^{n+1}$ 가 있을 때 $\textsf{ModSwitch} : \mathbb{Z}_Q^{n+1} \to \mathbb{Z}_q^{n+1}$ 는 다음과 같이 정의할 수 있다.
>
>\begin{align*}
>    \textsf{ModSwitch}((\vec{a}, b)) = ([a_0]_{Q:q}, [a_1]_{Q:q}, \dots, [a_{n-1}]_{Q:q}, [b]_{Q:q}).
>\end{align*}
>
>여기서 $([x]_{Q:q}) = \lfloor q/Q \cdot x \rceil \in \mathbb{Z}_q$ 이다.

코드를 통해 확인해보자.

In [5]:
def ModSwitch(ctxt:LWEctxt, Q:int, q:int) -> LWEctxt:
    a, b = ctxt
    mod_switched_a = [int(np.round((q/Q) * _a)) for _a in a]
    mod_switched_b = int(np.round((q/Q) * b))

    return (mod_switched_a, mod_switched_b)

In [6]:
n   = 32
q   = 512
Q   = 2048
std = 3.2

lwe_q = LWE(n, q, s_std="Gaussian", e_std=std)
lwe_Q = LWE(n, Q, s_std="Gaussian", e_std=std)

m   = 200
s   = lwe_q.keygen() # 버릴 비밀키
ct  = lwe_Q.encrypt(m, s)

mod_switched_ct = ModSwitch(ct, Q, q)
ptxt            = lwe_q.decrypt(mod_switched_ct, s)

print("Ideal Result : ", int(q/Q * m))
print("Plaintext    : ", ptxt)

Ideal Result :  50
Plaintext    :  438


매우 큰 노이즈를 보이는 것을 알 수 있다. 실제로 FHEW 에서 가장 많인 노이즈를 발생시키는 연산이다.

---

## Summary

이번에는 FHEW 에서 사용하는 부수적인 알고리즘인 key switch 와 modulus switch 에 대해서 알아보았다.

Key switching 과 달리 modulus switching 은 매우 큰 노이즈를 발생시키는 것을 알 수 있었으며, 실제로 FHEW 에서 수행하는 연산들 중 가장 많인 노이즈를 발생시키는 연산이다.

---

## Code implementation

`note_include/FHEW.py` 를 확인하면 위에서 정의한 연산들이 있는 Key siwtch and Modulus switch 의 구현을 확인할 수 있다.